In [3]:
# Importar las liobrerías y paquetes
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout


In [4]:
base_path = 'deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/'

In [5]:
# Inicializar la CNN
classifier = Sequential()

# Paso 1 - Seccion Convolución
classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), input_shape = (64, 64, 3), activation = "relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.25))

# Una segunda capa de convolución
classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = "relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.25))

# Una tercera capa de convolución
classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.25))

# Paso 3 - Flattening
classifier.add(Flatten())

# Paso 4 - Full Connection
classifier.add(Dense(units = 512, activation = "relu"))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 1, activation = "sigmoid"))

# Compilar la CNN
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 62, 62, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 29, 29, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)       

In [6]:
# Parte 2 - Ajustar la CNN a las imágenes para entrenar 
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        #rescale=1./255,
        #shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True
        rotation_range=15,
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1
)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory(base_path + 'dataset/training_set',
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='binary')

testing_dataset = test_datagen.flow_from_directory(base_path + 'dataset/test_set',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')



Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
# Definir callback para reducir learning rate

import os
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR) :
    os.mkdir(MODEL_DIR)
modelpath = './model/dogcat-{epoch:02d}-{val_loss:.4f}.hdf5'

checkPointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
earlystop = EarlyStopping(patience=10 )
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

callbacks=[earlystop, learning_rate_reduction, checkPointer]

In [8]:
# Entrenar

classifier.fit_generator(
                        training_dataset,
                        steps_per_epoch=8000,
                        epochs=30,
                        validation_data=testing_dataset,
                        validation_steps=2000,
                        callbacks=callbacks,
                        use_multiprocessing=True,
                        workers=16)

#classifier.save('model_dogcat_03')

Epoch 1/30
8000/8000 [==============================] - 289s 36ms/step - loss: 0.4502 - accuracy: 0.7851 - val_loss: 0.3599 - val_accuracy: 0.8198

Epoch 00001: val_loss improved from inf to 0.35991, saving model to ./model/dogcat-01-0.3599.hdf5
Epoch 2/30
8000/8000 [==============================] - 287s 36ms/step - loss: 0.3133 - accuracy: 0.8633 - val_loss: 0.1980 - val_accuracy: 0.8884

Epoch 00002: val_loss improved from 0.35991 to 0.19799, saving model to ./model/dogcat-02-0.1980.hdf5
Epoch 3/30
8000/8000 [==============================] - 287s 36ms/step - loss: 0.2714 - accuracy: 0.8838 - val_loss: 0.2879 - val_accuracy: 0.8857

Epoch 00003: val_loss did not improve from 0.19799
Epoch 4/30
8000/8000 [==============================] - 285s 36ms/step - loss: 0.2409 - accuracy: 0.8985 - val_loss: 0.1736 - val_accuracy: 0.9139

Epoch 00004: val_loss improved from 0.19799 to 0.17361, saving model to ./model/dogcat-04-0.1736.hdf5
Epoch 5/30
8000/8000 [==============================] -

In [11]:
# Parte 3 - Cómo hacer nuevas predicciones
import numpy as np
from keras.preprocessing import image
from tensorflow import keras


# Load model
classifier = keras.models.load_model('./model/dogcat-09-0.0205.hdf5')


test_image = image.load_img(base_path + 'dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
print(training_dataset.class_indices)
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

{'cats': 0, 'dogs': 1}
dog


In [15]:
STEP_SIZE_TEST = testing_dataset.n // testing_dataset.batch_size
result = classifier.evaluate_generator(generator=testing_dataset, steps=STEP_SIZE_TEST)
print("Accuracy = ",result[1])

  ...
    to  
  ['...']
Accuracy =  0.9148185
